Extraccion de palabras clave en las paginas web de la carpeta train:

Se realiza la extraccion de las palabras clave de todas las paginas web por clase, primero usando Beautiful Soup para la extraccion del contenido y luego Keybert para la extraccion de las palabras clave.

In [ ]:
import os
from bs4 import BeautifulSoup
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from keybert import KeyBERT
import json

El siguiente codigo realiza lo siguiente:

1. Se define una funcion que extrae el texto de cada pagina web usando Beautiful Soup.
2. Se procesan todas las paginas web de cada clase y se guarda el texto extraido en una lista.

In [30]:

# Ruta de la carpeta que contiene los archivos origen
folder_path = "C:/Users/usuario/Downloads/dataset_splitted/splits/train/student"  

# Lista creada para almacenar el texto del cuerpo de todas las páginas
all_page_texts = []

# Función para extraer el texto del cuerpo de la página
def extract_body_text(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Se analiza el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')

    # Se extrae el texto del cuerpo de la página
    page_text = ' '.join(soup.stripped_strings)
    page_text = page_text.replace('\n', '')

    return page_text

# Se procesan los archivos y se guarda el texto del cuerpo en la lista all_page_texts
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):  
        page_text = extract_body_text(file_path)
        all_page_texts.append(page_text)


En esta parte se usa Keybert para poder generar las palabras clave de todos los textos extraidos de las paginas web y que se almacenaron en la lista.

In [31]:
#Se usa keybert para obtener las palabras clave de todos los textos.
modelo = KeyBERT()
palabras_clave = modelo.extract_keywords(all_page_texts, stop_words='english')

#Ver las palabras clave encntradas
for keyword in palabras_clave:
    print(keyword)
    
todas_palabras_clave = [clave for lista_p_claves in palabras_clave for clave in lista_p_claves]

[('infrastructures', 0.3483), ('distributed', 0.3425), ('opal', 0.3412), ('projects', 0.3397), ('applications', 0.3232)]
[('sule', 0.5873), ('oumlzev', 0.4897), ('ozev', 0.4404), ('htmllast', 0.3277), ('1996', 0.327)]
[('dispersing', 0.5136), ('dispersion', 0.4719), ('bandwidth', 0.4567), ('fragmentation', 0.3769), ('utilization', 0.351)]
[('cache', 0.4833), ('performance', 0.358), ('locality', 0.3339), ('multiprocessors', 0.3302), ('memory', 0.3143)]
[('database', 0.4116), ('computing', 0.3912), ('alberta', 0.3303), ('muslim', 0.3092), ('iqbal', 0.2995)]
[('crl', 0.4164), ('masters', 0.383), ('ncsa', 0.372), ('graduate', 0.3696), ('professional', 0.3549)]
[('writings', 0.5208), ('essayist', 0.5163), ('write', 0.4796), ('wrote', 0.3847), ('poetry', 0.3831)]
[('bobby', 0.4555), ('phd', 0.4004), ('krupczak', 0.3876), ('telecommunications', 0.386), ('communications', 0.3847)]
[('moten', 0.568), ('cern', 0.4449), ('htmlcontent', 0.388), ('mime', 0.3851), ('roderick', 0.3122)]
[('keselman',

Para poder tener la palabra una sola vez, se calcula la suma del valor de cada palabra cuando esta varias veces, para esto se usa un diccionario.

In [32]:
# Se usa un diccionario para poder calcular la suma del valor de cada palabra clave cuando se encuentra varias veces.
sums = {}

# Se itera sobre las palabras clave encontradas y se suman sus valores
for clave in todas_palabras_clave:
    palabra, valor = clave  
    if palabra in sums:
        sums[palabra] += valor
    else:
        sums[palabra] = valor

# Se ordenan las palabras clave encontradas por el resultado de sus sumas de manera descendente
suma_ordenada = sorted(sums.items(), key=lambda x: x[1], reverse=True)

# Imprimir las palabras clave con sus sumas
for palabra, suma in suma_ordenada:
    print(f"Palabra clave: {palabra}, Suma: {suma}")


Palabra clave: cs, Suma: 62.18449999999998
Palabra clave: programming, Suma: 43.92000000000001
Palabra clave: computing, Suma: 38.421199999999985
Palabra clave: resume, Suma: 29.2804
Palabra clave: web, Suma: 24.890500000000007
Palabra clave: gmtserver, Suma: 23.131799999999995
Palabra clave: edu, Suma: 22.903700000000008
Palabra clave: page, Suma: 22.086300000000005
Palabra clave: university, Suma: 21.511500000000005
Palabra clave: computer, Suma: 20.807999999999996
Palabra clave: htmllast, Suma: 18.6883
Palabra clave: java, Suma: 17.7393
Palabra clave: apache, Suma: 17.619999999999997
Palabra clave: ncsa, Suma: 17.2273
Palabra clave: htmlcontent, Suma: 17.051900000000003
Palabra clave: student, Suma: 15.987200000000001
Palabra clave: mime, Suma: 15.865499999999997
Palabra clave: text, Suma: 15.792700000000002
Palabra clave: contact, Suma: 13.465299999999996
Palabra clave: gmtcontent, Suma: 13.194100000000002
Palabra clave: multimedia, Suma: 12.398899999999998
Palabra clave: distribut

Por ultimo se genera una lista ordenada con todas las palabras clave de cada clase y se guardan en un archivo JSON.

In [33]:
# Se crea una lista para todas las palabras clave
palabras_clave_lista = [palabra for palabra, suma in suma_ordenada]

# Ruta de la carpeta de salida para el archivo JSON
output_folder = "C:/Users/usuario/Downloads/dataset_splitted/splits/train_json/student_json"
os.makedirs(output_folder, exist_ok=True)

# Ruta completa al archivo JSON en la carpeta de salida
output_json_file = os.path.join(output_folder, "keywords.json")

# Se crea un diccionario para almacenar las palabras y el ground_truth
data_json = {"words": palabras_clave_lista, "ground_truth": "student"}

# Se guarda el diccionario en un archivo JSON 
with open(output_json_file, 'w', encoding='utf-8') as json_file:
    json.dump(data_json, json_file, ensure_ascii=False, separators=(',', ':'))

print(f"La lista de palabras clave se ha guardado en '{output_json_file}'")



La lista de palabras clave se ha guardado en 'C:/Users/usuario/Downloads/dataset_splitted/splits/train_json/student_json\keywords.json'


In [36]:
all_page_texts[2]

'Date: Thu, 21 Nov 1996 21:53:14 GMTServer: NCSA/1.5Content-type: text/html Boston Univeristy - Computer Science Dept. - Ph.D Candidate: Keith Kim, Gitae Kim, or Gi Tae Kim : TCP Boston, TCP-Boston K EITH (G ITAE ) K IM kgtjan@cs.bu.edu Welcome aboard!    I am a Ph.D candidate at Computer Science Dept. , Boston University .   I am currently a research assistant for Prof. Azer Bestavros , who is also my thesis advisor. Research Philosophy & Domain My three beliefs ... I believe in `resource dispersion\' to improve system performancein networked computing environment.  There are other good reasonsin dispersing systems resource, but I am particularly interested in minimizing the bottleneck effect in networked computing environment through load dispersion. I believe in thephrase, "The Simpler, The Faster", especially for the switch (router) architectures.  The higher the network bandwidth, the firmer I believe in it -- it does not make much sense to me that, messages stay longer in the swi